<a href="https://colab.research.google.com/github/Kerriea-star/TensorFlow-Customization/blob/main/Customization_basics_tensors_and_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here, you'll learn how to:

*   Import the required package.
*   Create and use tensors.
*   Use GPU acceleration.
*   Build a data pipeline with `tf.data.Dataset`



### Import TensorFlow

As of TensorFlow 2, eager execution is turned on by default. Eager execution enables a more interactive frontend to TensorFlow.

In [1]:
import tensorflow as tf

## Tensors

A Tensor is a multi-dimensional array. Similar to Numpy `ndarray` objects, `tf.Tensor` objects have a data type and a shape. Additionally, `tf.Tensor`s can reside in accelerator memory (like a GPU). TensorFlow offers a rich library of operations (for example, `tf.math.add`, `tf.linalg.matmul` and `tf.linalg.inv`) that consume and produce `tf.Tensor`s. These operations automatically convert built-in Python types. For example:

In [2]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


Each `tf.Tensor` has a shape and a datatype:

In [3]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


The most obvious differences between Numpy arrays and `tf.Tensor`s are:

1.   Tensors can be backed by accelerator memory(like GPU, TPU)
2.   Tensors are immutable

### NumPy compatibility

Converting between a `tf.Tensor` and a NumPy `ndarray` is easy:

*   TensorFlow operations automatically convert Numpy ndarrays to Tensors.
*   NumPy operations automatically convert Tensors to Numpy ndarrays.

Tensors are explicitly converted to NumPy ndarrays using their `.numpy()` method. These conversions are typically cheap since the array and the `tf.Tensor` share the underlying memory representation, if possible. However, sharing the underlying representation isn't always possible since the `tf.Tensor` may be hosted in GPU memory while the NumPy arrays are always backed by the host memory, and the conversion involves a copy from GPU to host memory.



In [5]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor  = tf.math.multiply(ndarray, 42)
print(tensor)

print("Add NumPy operations convert Tensors to a numpy array automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
Add NumPy operations convert Tensors to a numpy array automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU acceleration

Many TensorFlow operations are accelerated using the GPU for computation. Without any annotations, TensorFlow automatically decides whether to us the GPU or CPU for an operations-copying the tensor between CPU and GPU memory, if necessary.
Tensors produces by an operation are typically backed by the memory of the device on which the operation executed. For example:

In [6]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available:"),
print(tf.config.list_physical_devices("GPU"))

print("IS the Tensor on GPU #0: "),
print(x.device.endswith("GPU:0"))

Is there a GPU available:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
IS the Tensor on GPU #0: 
True


#### Device names

The `Tensor.device` property provides a fully qualified string name of the device hosting the contents of the tensor. This name encodes many details, such as an identifier of the network address of the host on which this program is executing and the device within that host. This is required for distributed execution of a TensorFlow program. The string ends with `GPU:<N>` if the tensor is placed on the N-th GPU on the host.

#### Explicit device placement

In TensorFlow, placement refers to how individual operations are assigned (placed on) a device for execution. As mentioned, when there is no explicit guidance provided, TensorFlow automatically decides which device to execute an operation and copies tensors to that device, if needed.

However, TensorFlow operations can be explicitly placed on specific devices using the `tf.device` context manager. For example:

In [7]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the send GPU etc
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 299.67ms
On GPU:
10 loops: 3794.32ms
